# ***WaZP catalogs notebook***

Author: Gabriel Luan Souza de Oliveira.

Last update: 07/03/2022.

## ***Imported modules***

In [ ]:
%load_ext autoreload
%autoreload 2

#import numpy as np
#import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from astropy.table import Table
#import healpy as hp
#import copy

#from gavodb import DBManager
#import sqlalchemy

## ***Loading data***

In [ ]:
clusters_table_path = '/archive/user/aguena/cats/dc2/cosmoDC2_v1.1.4/extragal/wazp/6563/wazp_cluster.fits'
members_table_path = '/archive/user/aguena/cats/dc2/cosmoDC2_v1.1.4/extragal/wazp/6563/wazp_membership.fits'
clusters_table = Table.read(clusters_table_path)
members_table = Table.read(members_table_path)

## ***Inspecting data***

In [ ]:
def inspect_data(clusters_table,members_table):
    
    clusters_n = len(clusters_table)
    members_n = len(members_table)
    print('Número de clusters: ', clusters_n, '\n')
    print('Número de members: ', members_n, '\n')
    
    cluster_keys = clusters_table.keys()
    members_keys = members_table.keys()
    
    print('Keywords de clusters: \n', cluster_keys, '\n')
    print('Keywords de members: \n', members_keys, '\n')
    
    print(clusters_table[0:10])
    print(members_table[0:10])
inspect_data(clusters_table,members_table)

In [ ]:
def inspect_columns(table, columns):
    print(table[columns])
    return table[columns]
#inspect_columns(clusters_table, ['SeqNr', 'ID'])
#inspect_columns(members_table, ['SeqNr', 'ID_CLUSTER'])

## ***Selections***

### Ngals selection

In [ ]:
clusters_mask = clusters_table['NGALS'] >= 200
clusters_table_reduced = clusters_table[clusters_mask]

members_mask = np.isin(members_table['ID_CLUSTER'],clusters_table_reduced['ID'])
members_table_reduced = members_table[members_mask]

print('Number of NGALS>200 clusters: ', len(clusters_table_reduced))
print('Number of members associated to NGALS>200 clusters: ', len(members_table_reduced))

### Color choice

In [ ]:
bands = ['u','g','r','i','z','y','j','h','k']
aux_bands = bands[1:]+bands[:1]
colors_keywords = [i+'-'+j for i,j in zip(bands,aux_bands)]
colors_keywords.pop(-1)

mag_keywords = ['mag_u', 'mag_g', 'mag_r', 'mag_i', 'mag_z', 'mag_y', 'mag_j', 'mag_h', 'mag_k']
aux_mag_keywords = mag_keywords[1:]+mag_keywords[:1]
mag_keywords.pop(-1)
aux_mag_keywords.pop(-1)

for i,j,k in zip(colors_keywords,mag_keywords,aux_mag_keywords):
    members_table_reduced[i] = members_table_reduced[j] - members_table_reduced[k]

#print(members_table_reduced[colors_keywords])
print(members_table_reduced[mag_keywords])

### CMD

In [ ]:
clusters_dict = {}
for each in clusters_table_reduced['ID']:
    cluster_mask = members_table_reduced['ID_CLUSTER'] == each
    clusters_dict['{}'.format(each)] = members_table_reduced[cluster_mask]
    
#plot

plt.rcParams['axes.facecolor'] = 'white'
band = 3
for cluster_id in clusters_table_reduced['ID'].astype(str):
    color = clusters_dict[cluster_id][colors_keywords[band]]
    mask = color <= 500
    mag = clusters_dict[cluster_id][mag_keywords[band+1]]
    redshift = clusters_dict[cluster_id]['ZP']
    plt.scatter(mag[mask], color[mask], s = 1.0, marker = ',', c = redshift[mask], cmap = cm.hot_r)
    cbar = plt.colorbar()
    cbar.set_label('Redshift')
    plt.xlabel(mag_keywords[band+1])
    plt.ylabel(colors_keywords[band])
    plt.show()
    plt.clf()


## ***Data information***

1) Cluster's ID goes from <b>1</b> to <b>40389</b>.

2) Ngals is different from the number of members related to a specific id_cluster

In [ ]:
#1)
print(np.nanmin(inspect_columns(members_table, 'ID_CLUSTER')),
      np.nanmax(inspect_columns(members_table, 'ID_CLUSTER')))

In [ ]:
#2)
print(clusters_table_reduced['ID','NGALS'])
for each in clusters_table_reduced['ID']:
    print(each,len(members_table_reduced['ID_CLUSTER'][members_table_reduced['ID_CLUSTER'] == each]))